#Mount del drive e installazione/importazione librerie necessarie

In [240]:
#!pip install torch torchvision
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
percorso =  "/content/drive/MyDrive/Progetto_laboratorio"
#%cd /content/drive/MyDrive/Progetto_laboratorio
#!ls
import sys
sys.path.append(percorso)
import fiona
import os


#!pip install rasterio
#!pip install geopandas
#!pip install matplotlib
#!pip install albumentations
#!pip install tqdm

import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF


Mounted at /content/drive


In [267]:
import importlib
import dataset
importlib.reload(dataset)
import model_unet
importlib.reload(model_unet)
import utils
importlib.reload(utils)
from dataset import Dataset
from model_unet import UNET
from utils import (
    save_checkpoint,
    load_checkpoint,
    check_precision,
    check_recall,
    check_f1,
    check_accuracy,
    check_metrics,
    get_loaders,
)

In [233]:
#Hyperparameters
LEARNING_RATE = 1e-2
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 32
NUM_EPOCHS = 10
NUM_WORKERS = 2
#IMAGE_HEIGHT = 256
#IMAGE_WIDTH = 256
PIN_MEMORY = True
LOAD_MODEL = False
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TRAIN_DIR = percorso + '/Dataset/train'
VAL_DIR = percorso + "/Dataset/validation"
TEST_DIR = percorso + "/Dataset/test"



In [241]:
import random
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)


#Training modello

In [260]:
def train_fn(loader, model, optimizer, loss_fn, scaler):
  loop = tqdm(loader)                                       #progress bar

  for batch_idx, (data, targets) in enumerate(loop):                       #iteriamo sui batch forniti dal loader
    data = data.to(device = DEVICE).permute(0,3,1,2)                                       #sposta i dati sul device , CPU o GPU
    targets = targets.float().unsqueeze(1).to(device = DEVICE)             #convertiamo targets in float e aggiungiamo una dimensione extra
    print(data.shape)
    print(targets.shape)
    # forward
    with torch.cuda.amp.autocast():                                        #abilita il mixed precision training
      predictions = model(data)                                            #calcoliamo predizioni
      loss = loss_fn(predictions, targets)                                 #calcoliamo la loss

    # backward
    optimizer.zero_grad()                                                  #azzera i gradienti dell'optimizer
    scaler.scale(loss).backward()                                          #scala la loss e calcola i gradienti con backward
    scaler.step(optimizer)                                                 #aggiorna i parametri
    scaler.update()                                                        #aggiorna lo scaler

    # update tqdm loop
    loop.set_postfix(loss = loss.item())                                     #aggiorna il progress bar con il valore della loss





#Prova del modello

In [265]:
def main():
  train_transform = A.Compose([
      A.Rotate(limit=35, p=1.0),
      A.HorizontalFlip(p=0.5),
      A.VerticalFlip(p=0.1),
      A.Normalize(
          mean=[0.0, 0.0, 0.0],
          std=[1.0, 1.0, 1.0],
          max_pixel_value=255.0,
      ),
      ToTensorV2(),
  ], is_check_shapes=False)

  val_transform = A.Compose([
      A.Normalize(
          mean=[0.0, 0.0, 0.0],
          std=[1.0, 1.0, 1.0],
          max_pixel_value=255.0,
      ),
      ToTensorV2(),
  ], is_check_shapes=False)

  model = UNET(in_channels=6, out_channels=1).to(device = DEVICE)
  loss_fn = nn.BCEWithLogitsLoss()
  optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
  scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=0.9*NUM_EPOCHS)                 #riduce il lr di 0.1 ad intervalli regolari
  scaler = torch.cuda.amp.GradScaler()
  set_seed(42)

  train_loader, val_loader = get_loaders(percorso, BATCH_SIZE, train_transform, val_transform, num_workers=2, pin_memory=PIN_MEMORY)
  scaler = torch.cuda.amp.GradScaler()

  if LOAD_MODEL:
    load_checkpoint(torch.load("model.pth.tar"), model)

  all_predictions = []
  all_targets = []
  print("Training")

  for epoch in range (NUM_EPOCHS):
    train_fn(loader=train_loader, model=model, optimizer=optimizer, loss_fn=loss_fn, scaler=scaler)

    for inputs,targets in val_loader:
      print(inputs.shape)
      print(targets.shape)
      inputs, targets = (inputs.permute(0,3,1,2)).to(DEVICE),targets.to(DEVICE)
      outputs = model(inputs.permute(0, 3, 1, 2))


      all_predictions.append(outputs.cpu().detach().numpy())
      all_targets.append(targets.cpu().detach().numpy())

  check_metrics(all_targets, all_predictions)


  scheduler.step()


In [266]:

if __name__ == "__main__":
  try:
      torch.multiprocessing.set_start_method('spawn', force=True)
  except RuntimeError:
      pass
  main()


Training


  0%|          | 0/2 [00:00<?, ?it/s]

torch.Size([32, 3, 2048, 1024])
torch.Size([32, 1, 1024, 1024])


  0%|          | 0/2 [01:12<?, ?it/s]


RuntimeError: Given groups=1, weight of size [64, 6, 3, 3], expected input[32, 3, 2048, 1024] to have 6 channels, but got 3 channels instead